# Assignment 3 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Personal Details:

In [1]:
# Details Student 1:
# Name: Bar Amir
# ID: 208754580
# Details Student 2:
# Name: Boaz Yunayev
# ID: 322271271


## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [2]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [3]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [4]:
# word net installation:

# unmark if you want to use and need to install
%pip install wn
%python -m wn download omw-he:1.4

Note: you may need to restart the kernel to use updated packages.


UsageError: Line magic function `%python` not found (But cell magic `%%python` exists, did you mean that instead?).


In [5]:
# word net import:

# unmark if you want to use:
import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [6]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
%pip install hebrew_tokenizer

Note: you may need to restart the kernel to use updated packages.


In [7]:
# Hebrew tokenizer import:

# unmark if you want to use:
import hebrew_tokenizer as ht

c:\Users\boazy\MainOneDrive\Desktop\Angular\assign3


### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [8]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [9]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [10]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

In [11]:
# Tokenizing

def my_tokenizer(hebrew_text):
    tokens = [token[1] for token in ht.tokenize(hebrew_text) if token[0] == 'HEBREW']
    return tokens

In [12]:
# Mapping female and male to numerical values

df_train['gender'] = np.where(df_train['gender'] == 'm',1,0)

In [13]:
# Split the training dataset to feature vectors and target label

X_train = df_train['story']
y_train = df_train['gender']

# **Cross Validation**

In [14]:
# Define your pipelines and parameter grids as you did before

SGDClassifier_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=my_tokenizer)),
    ('normalize', preprocessing.Normalizer(norm='l2')),
    ('clf', SGDClassifier(max_iter=1000, random_state=42))
])

NB_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=my_tokenizer)),
    ('normalize', preprocessing.Normalizer(norm='l2')),
    ('clf', MultinomialNB())
])

svc_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=my_tokenizer)),
    ('normalize', preprocessing.Normalizer(norm='l2')),
    ('classifier', LinearSVC())
])

mlp_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=my_tokenizer)),
    ('normalize', preprocessing.Normalizer(norm='l2')),
    ('classifier', MLPClassifier())
])

SGDClassifier_param_grid = {
    'tfidf__max_df': [0.5, 0.75, 1.0],
    'clf__alpha': [0.0001, 0.001, 0.01, 0.1],
    'clf__penalty': ['l1', 'l2', 'elasticnet'],
    'clf__max_iter': [1000, 2000, 3000]
}

NB_param_grid = {
    'tfidf__max_df': [0.5, 0.75, 1.0],
    'clf__alpha': [0.0001, 0.001, 0.01, 0.1]
}

svc_param_grid = {
    'tfidf__max_df': [0.5, 0.75, 1.0],
    'classifier__C': [0.1, 1, 10],
    'classifier__penalty': ['l1', 'l2'],
    'classifier__max_iter': [1000, 2000, 3000]
}

mlp_param_grid = {
    'tfidf__max_df': [0.5, 0.75, 1.0],
    'classifier__hidden_layer_sizes': [(50,), (100,)],
    'classifier__activation': ['logistic', 'relu'],
    'classifier__solver': ['sgd', 'adam'],
    'classifier__max_iter': [1000, 2000]
}

# **Best model**

In [15]:
# Find the best params for SGDClassifier

SGDClassifier_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=my_tokenizer,max_df=0.73,ngram_range=(1,1))),
    ('normalize', preprocessing.Normalizer(norm='l2')),
    ('clf', SGDClassifier(random_state=42))
])

param_grid = {
    'clf__alpha': [0.0001, 0.001],
    'clf__penalty': ['l1', 'l2'],
    'clf__loss': ['hinge', 'log'],
    'clf__max_iter': [10,22,50],
    'clf__learning_rate': ['constant', 'optimal'],
    'clf__eta0': [0.1, 1, 10],
    'clf__validation_fraction': [0.1, 0.2],
    'clf__n_iter_no_change': [3, 6]
}


# Perform GridSearch
grid_search = GridSearchCV(SGDClassifier_pipeline,
                           param_grid, cv=10, scoring='f1_macro')
grid_search.fit(X_train, y_train)

# Print the best hyperparameters and cross-validation score
print("Best hyperparameters:", grid_search.best_params_)
print("Cross-validation F1 score:", grid_search.best_score_)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(max_df=0.73,
                                                        tokenizer=<function my_tokenizer at 0x0000027656B277E0>)),
                                       ('normalize', Normalizer()),
                                       ('clf',
                                        SGDClassifier(random_state=42))]),
             param_grid={'clf__alpha': [0.0001, 0.001],
                         'clf__eta0': [0.1, 1, 10],
                         'clf__learning_rate': ['constant', 'optimal'],
                         'clf__loss': ['hinge', 'log'],
                         'clf__max_iter': [10, 22, 50],
                         'clf__n_iter_no_change': [3, 6],
                         'clf__penalty': ['l1', 'l2'],
                         'clf__validation_fraction': [0.1, 0.2]},
             scoring='f1_macro')

Best hyperparameters: {'clf__alpha': 0.0001, 'clf__eta0': 0.1, 'clf__learning_rate': 'optimal', 'clf__loss': 'hinge', 'clf__max_iter': 22, 'clf__n_iter_no_change': 6, 'clf__penalty': 'l1', 'clf__validation_fraction': 0.1}
Cross-validation F1 score: 0.7207252744468724


### **Average f1 Score of Best Model with TFID**

In [16]:
best_model_tfidf = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=my_tokenizer,
                              max_df=0.73, ngram_range=(1, 1))),
    ('normalize', preprocessing.Normalizer(norm='l2')),
    ('clf', SGDClassifier(max_iter=22, random_state=42, alpha=0.0001, penalty='l1', loss='hinge',
                          learning_rate='optimal', eta0=1, validation_fraction=0.1, n_iter_no_change=6,))
])

# Assuming you already have X_train and y_train defined
best_model_tfidf.fit(X_train, y_train)

# Use cross_val_score to calculate the average F1 score
f1_scores = cross_val_score(
    best_model_tfidf, X_train, y_train, cv=10, scoring='f1_macro')
avg_f1_score = np.mean(f1_scores)
print("Average F1 score:", avg_f1_score)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_df=0.73,
                                 tokenizer=<function my_tokenizer at 0x0000027656B277E0>)),
                ('normalize', Normalizer()),
                ('clf',
                 SGDClassifier(eta0=1, max_iter=22, n_iter_no_change=6,
                               penalty='l1', random_state=42))])

Average F1 score: 0.7207252744468724


### **Average f1 Score of Best Model with CountVectorizer**

In [17]:
best_model_countvectorizer = Pipeline([
    ('countvectorizer', CountVectorizer(tokenizer=my_tokenizer,
                                        max_df=0.73, ngram_range=(1, 1))),
    ('normalize', preprocessing.Normalizer(norm='l2')),
    ('clf', SGDClassifier(max_iter=22, random_state=42, alpha=0.0001, penalty='l1', loss='hinge',
                          learning_rate='optimal', eta0=1, validation_fraction=0.1, n_iter_no_change=6,))
])

# Assuming you already have X_train and y_train defined
best_model_countvectorizer.fit(X_train, y_train)

# Use cross_val_score to calculate the average F1 score
f1_scores = cross_val_score(
    best_model_countvectorizer, X_train, y_train, cv=10, scoring='f1_macro')
avg_f1_score = np.mean(f1_scores)
print("Average F1 score:", avg_f1_score)

Pipeline(steps=[('countvectorizer',
                 CountVectorizer(max_df=0.73,
                                 tokenizer=<function my_tokenizer at 0x0000027656B277E0>)),
                ('normalize', Normalizer()),
                ('clf',
                 SGDClassifier(eta0=1, max_iter=22, n_iter_no_change=6,
                               penalty='l1', random_state=42))])

Average F1 score: 0.6866805870760653


In [18]:
# Predict using the best model
y_pred = best_model_tfidf.predict(df_test["story"])

# Map predicted values to 'm' and 'f'
y_pred = np.where(y_pred == 1, 'm', 'f')

# Create a DataFrame with 'id_test' and 'y_pred' columns
result_df = pd.DataFrame({'id_test': df_test["test_example_id"], 'y_pred': y_pred})

# Display the first 5 rows of the result
print(result_df.head(5))

# Display the last 5 rows of the result
print(result_df.tail(5))

   id_test y_pred
0        0      m
1        1      m
2        2      m
3        3      m
4        4      f
     id_test y_pred
318      318      m
319      319      m
320      320      m
321      321      m
322      322      m


In [19]:
# create dataframe with the test results
df_predicted = pd.DataFrame(
    {'test_example_id': df_test['test_example_id'], 'predicted_category': y_pred})
df_predicted

,test_example_id,predicted_category
0,0,m
1,1,m
2,2,m
3,3,m
4,4,f
...,...,...
318,318,m
319,319,m
320,320,m
321,321,m


### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [20]:
df_predicted.to_csv('classification_results.csv',index=False)